In [11]:
from pyexpat import model
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np
import glob
import cv2
import os
import locale

import tensorflow as tf
#import all necessary layers
from tensorflow.keras.layers import Input, DepthwiseConv2D
from tensorflow.keras.layers import Conv2D, BatchNormalization
from tensorflow.keras.layers import ReLU, AvgPool2D, Flatten, Dense
from tensorflow.keras import Model

from sklearn.model_selection import train_test_split
import matplotlib.image as mpimg
from tensorflow.keras.layers import concatenate
from deepforest import CascadeForestClassifier

In [2]:
gasdata = pd.read_csv("/Users/Eddie/Downloads/data5.csv")
data_dir = '/Users/Eddie/Downloads/Data5'
imagedata = sorted(os.listdir(data_dir))
print(len(imagedata))
X_data = []
for image in imagedata:
        # print(image)
        img = mpimg.imread('/Users/Eddie/Downloads/Data5/'+image)
        img = img.reshape(320,320,9)
        img = img/255.0
        X_data.append(img)
images = np.array(X_data)
print(images.shape)

1000
(1000, 320, 320, 9)


In [3]:
split = train_test_split(gasdata, images, test_size=0.2)
(trainAttrX, testAttrX, trainImagesX, testImagesX) = split
trainy = trainAttrX["Gas"]
testy = testAttrX["Gas"]
trainAttrX = trainAttrX.drop(columns=['Gas'])
testAttrX = testAttrX.drop(columns = ['Gas'])
trainAttrX= (trainAttrX - np.min(trainAttrX)) / (np.max(trainAttrX) - np.min(trainAttrX))
testAttrX = (testAttrX - np.min(testAttrX)) / (np.max(testAttrX) - np.min(testAttrX))
print(trainAttrX.shape)
print(testAttrX.shape)
print(trainy.shape)
print(testy.shape)

(800, 2)
(200, 2)
(800,)
(200,)


In [4]:
# MobileNet block
def mobilnet_block (x, filters, strides):
    
    x = DepthwiseConv2D(kernel_size = 3, strides = strides, padding = 'same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    x = Conv2D(filters = filters, kernel_size = 1, strides = 1)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    return x

In [5]:
def create_cnn_mobilenet():
    input = Input(shape = (320,320,9))
    x = Conv2D(filters = 32, kernel_size = 3, strides = 2, padding = 'same')(input)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    # main part of the model
    x = mobilnet_block(x, filters = 64, strides = 1)
    x = mobilnet_block(x, filters = 128, strides = 2)
    x = mobilnet_block(x, filters = 128, strides = 1)
    x = mobilnet_block(x, filters = 256, strides = 2)
    x = mobilnet_block(x, filters = 256, strides = 1)
    x = mobilnet_block(x, filters = 512, strides = 2)
    for _ in range (5):
        x = mobilnet_block(x, filters = 512, strides = 1)
    x = mobilnet_block(x, filters = 1024, strides = 2)
    x = mobilnet_block(x, filters = 1024, strides = 1)
    x = AvgPool2D (pool_size = 7, strides = 1, data_format='channels_first')(x)
   # output = Dense (units = 1000, activation = 'softmax')(x)
    output = Flatten()(x)
    model = Model(inputs=input, outputs=output)
    # model.summary()

    return model

In [7]:
mobilenet = create_cnn_mobilenet()
mobilenetPredict = mobilenet.predict(trainImagesX)
print(mobilenetPredict.shape)


25/25 [==============================] - 13s 499ms/step
(800, 40720)


In [9]:
mobilenetPredict = np.array(mobilenetPredict)
trainAttrX = np.array(trainAttrX)
print(mobilenetPredict.shape)
print(trainAttrX.shape)
trainData = concatenate([mobilenetPredict, trainAttrX])
trainData = np.array(trainData)
trainData.reshape(-1,1)

print(trainData.shape)

(800, 40720)
(800, 2)
(800, 40722)


In [12]:
model = CascadeForestClassifier(n_estimators=50,random_state=1)
model.fit(trainData,trainy)

[2023-12-17 20:39:59.708] Start to fit the model:
[2023-12-17 20:39:59.708] Fitting cascade layer = 0 
[2023-12-17 20:53:58.070] layer = 0  | Val Acc = 96.875 % | Elapsed = 838.363 s
[2023-12-17 20:53:58.270] Fitting cascade layer = 1 
[2023-12-17 20:58:12.123] layer = 1  | Val Acc = 97.750 % | Elapsed = 253.854 s
[2023-12-17 20:58:12.147] Fitting cascade layer = 2 
[2023-12-17 21:02:21.045] layer = 2  | Val Acc = 97.875 % | Elapsed = 248.898 s
[2023-12-17 21:02:21.051] Fitting cascade layer = 3 
[2023-12-17 21:06:17.794] layer = 3  | Val Acc = 97.875 % | Elapsed = 236.743 s
[2023-12-17 21:06:17.794] Early stopping counter: 1 out of 2
[2023-12-17 21:06:17.800] Fitting cascade layer = 4 
[2023-12-17 21:10:03.962] layer = 4  | Val Acc = 98.000 % | Elapsed = 226.162 s
[2023-12-17 21:10:03.976] Fitting cascade layer = 5 
[2023-12-17 21:14:10.320] layer = 5  | Val Acc = 98.000 % | Elapsed = 246.344 s
[2023-12-17 21:14:10.320] Early stopping counter: 1 out of 2
[2023-12-17 21:14:10.331] Fitt

In [3]:
#stem of the model
input = Input(shape = (320,320,9))
x = Conv2D(filters = 32, kernel_size = 3, strides = 2, padding = 'same')(input)
x = BatchNormalization()(x)
x = ReLU()(x)

In [4]:
# main part of the model
x = mobilnet_block(x, filters = 64, strides = 1)
x = mobilnet_block(x, filters = 128, strides = 2)
x = mobilnet_block(x, filters = 128, strides = 1)
x = mobilnet_block(x, filters = 256, strides = 2)
x = mobilnet_block(x, filters = 256, strides = 1)
x = mobilnet_block(x, filters = 512, strides = 2)
for _ in range (5):
     x = mobilnet_block(x, filters = 512, strides = 1)
x = mobilnet_block(x, filters = 1024, strides = 2)
x = mobilnet_block(x, filters = 1024, strides = 1)
x = AvgPool2D (pool_size = 7, strides = 1, data_format='channels_first')(x)
output = Dense (units = 1000, activation = 'softmax')(x)
model = Model(inputs=input, outputs=output)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 112, 112, 32)      896       
                                                                 
 batch_normalization (Batch  (None, 112, 112, 32)      128       
 Normalization)                                                  
                                                                 
 re_lu (ReLU)                (None, 112, 112, 32)      0         
                                                                 
 depthwise_conv2d (Depthwis  (None, 112, 112, 32)      320       
 eConv2D)                                                        
                                                                 
 batch_normalization_1 (Bat  (None, 112, 112, 32)      128   

In [5]:
#plot the model
tf.keras.utils.plot_model(model, to_file='model.png', show_shapes=True, show_dtype=False,show_layer_names=True, rankdir='TB', expand_nested=False, dpi=96)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.
